# **Mục lục**

**1. Giới thiệu & Mục tiêu**<br>
- 1.1 Bối cảnh dự án (Olist)<br>
- 1.2 Mục tiêu của Notebook này<br>

**2. Tải dữ liệu**<br>
- 2.1 Import thư viện<br>
- 2.2 Tải tất cả các bảng dữ liệu Olist và xem nhanh (.head())<br>

**3. Tổng quan & mô tả dữ liệu**<br>
- 3.1 ERD<br>
- 3.2 Thông tin các bảng: customers, orders, order_items, products, sellers, order_payments, order_reviews, geolocation, product_category_name_translation<br>
Kiểm tra thông tin chính: kích thước, kiểu dữ liệu, khóa chính<br>

**4. Chất lượng dữ liệu**<br>
- Quan sát chất lượng dữ liệu ban đầu: thiếu dữ liệu, trùng lặp, cột nghi vấn<br>

**5. Tìm hiểu các cột thời gian**<br>
- 5.1 Liệt kê các cột thời gian và chuyển sang datetime<br>
- 5.2 Kiểm tra logic thời gian (vd: giao hàng trước ngày mua?)<br>

**6. Hiểu dữ liệu từ góc nhìn nghiệp vụ**<br>
- 6.1 Olist hoạt động như thế nào (Marketplace)<br>
- 6.2 Ý nghĩa các bảng dữ liệu chính và các biến quan trọng cho phân tích<br>

**7. Tổng kết & Hướng đi tiếp theo**<br>
- 7.1 Tóm tắt phát hiện chính<br>
- 7.2 Danh sách việc cần làm trong 02_data_cleaning.ipynb<br>

# **1. Giới thiệu & Mục tiêu**

# **2. Tải dữ liệu**

# **3. Tổng quan & mô tả dữ liệu**

# **4. Chất lượng dữ liệu**

# **5. Tìm hiểu các cột thời gian**

# **6. Hiểu dữ liệu từ góc nhìn nghiệp vụ**

# **7. Tổng kết & Hướng đi tiếp theo**